# Face Shape Classification using MobileNetV2 and Tensorflow
![](https://www.researchgate.net/publication/342856036/figure/fig3/AS:911929400885251@1594432320422/The-architecture-of-the-MobileNetv2-network.ppm)

In order to download, I had to set an access token for Kaggle API and install needed python packages. Set-up kaggle access token json file, and allow full read and write access to the file, while no other user can access the file using chmod 600 code.

In [ ]:
# Setting up access to kaggle api
! printf '{"username":"razvantalexandru", "key":"d43f87acf66f1b73c65c17d74cae5f55"}' > /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json

# Download the dataset
! rm -rf dataset
! kaggle datasets download -d niten19/face-shape-dataset
! unzip face-shape-dataset.zip -d dataset
! mv dataset/'FaceShape Dataset'/training_set dataset/train
! mv dataset/'FaceShape Dataset'/testing_set dataset/test
! rm -rf dataset/'FaceShape Dataset'

In [ ]:
# Setting up access to kaggle api
! printf '{"username":"razvantalexandru", "key":"d43f87acf66f1b73c65c17d74cae5f55"}' > /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json

# Download the dataset
! rm -rf dataset
! kaggle datasets download -d niten19/face-shape-dataset
! unzip face-shape-dataset.zip -d dataset
! mv dataset/'FaceShape Dataset'/training_set dataset/train
! mv dataset/'FaceShape Dataset'/testing_set dataset/test
! rm -rf dataset/'FaceShape Dataset'

In [ ]:
# Define imports
import os
import tensorflow as tf
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
tf.config.list_physical_devices('GPU')

from PIL import Image
from tqdm import tqdm
from struct import unpack
from PIL import ImageFile
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.applications.inception_v3 import preprocess_input

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# Additional definitions
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Define config params and Hyperparams
DEBUG = False
EPOCHS = 100
LR = 1e-3
DATASET_SEED = 1337
IMAGE_SIZE = 224
BATCH_SIZE = 8
DS_FROM_DIR = False


# Define needed paths
BASE_DS_PATH = 'dataset'
TRAIN_SPLIT_PATH = os.path.join(BASE_DS_PATH, 'train')
EVAL_SPLIT_PATH = os.path.join(BASE_DS_PATH, 'test')
CHKPT_FILEPATH = 'checkpoints/best.model'

# --------- Setup classes -----------
classes = {
    0: 'Heart',
    1: 'Oblong',
    2: 'Oval',
    3: 'Round',
    4: 'Square'
}

NUM_CLASSES = len(classes.keys())

In [ ]:
#----------CHECK DATASET FOR INCONSISTIENCES----------
class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]            
            if len(data)==0:
                break   
# JPEG binary marker mappings (verify integrity of jpeg file)
marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


assert os.path.exists(BASE_DS_PATH), 'Given path does not exist. Provide a valid one'
corrupted = []

for root, subdirectories, files in tqdm(os.walk(BASE_DS_PATH)):
    for im_file in files:
        file_path = os.path.join(root, im_file)
        try:
            image = JPEG(file_path) 
            image.decode()   
        except:
          # Image is corrupted -> removing
            if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Removed {file_path} - Corrupted image")

In [ ]:
if not DS_FROM_DIR:
    # Setup data-generators
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale = 1./255., 
        rotation_range = 40, 
        zoom_range = 0.2, 
        horizontal_flip = True
    )

    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
       rescale = 1./255.
    )
    

    # Flow from directory data loaders
    train_generator = train_datagen.flow_from_directory(
        TRAIN_SPLIT_PATH,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        target_size=(IMAGE_SIZE, IMAGE_SIZE))
    valid_generator = valid_datagen.flow_from_directory(
        EVAL_SPLIT_PATH,
        class_mode='categorical',
        batch_size=BATCH_SIZE,
        target_size=(IMAGE_SIZE, IMAGE_SIZE))

else:
    # Setup data-loaders using DS_FROM_DIR
    train_generator = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_SPLIT_PATH,
        validation_split=0.1,
        subset="training",
        seed=DATASET_SEED,
        image_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
    )

    valid_generator = tf.keras.preprocessing.image_dataset_from_directory(
        EVAL_SPLIT_PATH,
        validation_split=0.2,
        subset="validation",
        seed=DATASET_SEED,
        image_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
    )

if DEBUG:
  # Visualize a few imagess
  plt.figure(figsize=(10, 10))
    for images, labels in train_ds.take(1):
        for i in range(100):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(int(labels[i]))
            plt.axis("off")

In [ ]:
# Select way to construct dataset, using flow-from-dir or image-dataset
if DS_FROM_DIR:
    # Apply augmentations over data loader
    train_dataset = train_dataset.map(
      lambda x, y: (augmentation_pipeline(x, training=True), y))
    
        # Convert generators to dataset loaders
    train_dataset = tf.data.Dataset.from_generator(train_generator)
    valid_dataset = tf.data.Dataset.from_generator(valid_generator)

    # Prefetch data
    train_dataset = train_dataset.prefetch(buffer_size=BATCH_SIZE)
    valid_dataset = valid_dataset.prefetch(buffer_size=BATCH_SIZE)


# Compile model with sparse-categorical (multi-class)
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=["accuracy"],
)

history = model.fit(
  train_generator,
  validation_data=valid_generator,
  epochs=EPOCHS,
  callbacks=[chkpt_callback]
)

model.save("saved_model")

In [ ]:
# See training stats
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()